# 第14章 C++中的代码重用

## 14.1 包含对象成员的类

### 14.1.1 valarray类简介

valarray类是由头文件 valarray 支持的。定义了数组中的方法，包括所有元素相加、最大值和最小值等。

### 14.1.2 Student类的设计

### 14.1.1 Student类示例

---

## 14.2 私有继承

包含将对对象作为一个命名的成员对象，而私有继承将对象作为一个未被命名的继承对象添加到类中。

### 14.2.1 Student类示例（新版本）

private是默认值，因此省略访问限定符也将导致私有继承。

#### 1. 初始化基类组件

包含将使用这样的构造函数：
```cpp
Student(const char * str, const double * pd, int n)
    : name(str), scores(pd, n) {}
```

继承类，使用类名而不是成员名来标识构造函数
```cpp
Student(const char * str, const double * pd, int n)
    : std::string(str), ArrayDb(pd, n) {}
```

#### 2. 访问基类的方法

使用包含时，将使用对象名来调用方法，而使用私有继承时将使用类名和作用域解析符来调用方法。

#### 3. 访问基类对象

使用作用域解析符可以访问基类的方法，但如果要使用基类对象本身，该怎么做呢？例如，Student类的包含版本实现了Name()方法，返回string对象成员name；但使用私有继承时，该string对象没有名称。那么，Student类的代码该如何访问内部的string对象呢？

答案是强制类型转换。由于Student类是由string类派生而来的，因此可以通过强制类型转换，将Student对象转换为string对象。

```cpp
const string& Student::Name() const
{
    return (const string&) *this;
}
```

#### 4. 访问基类的友元函数

可以使用显式地转换为基类来调用正确的函数
```cpp
ostream& operator<<(ostream* os, const Student& stu)
{
    os << "Scores for " << (const string&) stu << ":\n";
}
```

如果不使用类型转换，下述代码将与友元函数原型匹配，从而导致递归调用。
```cpp
os << stu;
```

### 14.2.2 使用包含还是私有继承

### 14.2.3 保护继承

隐式向上转换(implicit upcasting)意味着无需进行显式类型转换，就可以将基类指针或引用指向派生类对象。
1. 公有继承可以进行隐式向上转换
2. 保护继承只能在派生类中进行隐式向上转换
3. 私有类型不能支持隐式向上转换

### 14.2.4 使用using重新定义访问权限

```cpp
class Student : private std::string, private std::valarray<double>
{
public:
    using std::valarray<double>::min;
    using std::valarray<double>::max;
}
```

上述using声明使得`valarray<double>::min()`和`valarray<double>::max()`可用，就像它们是Student的公有方法一样

```cpp
cout << "high score: " << ada[i].max() << endl;
```

注意，using声明只使用成员名--没有圆括号、函数特征标和返回类型。例如，为使Student类可以使用valarray的`operator[]()`方法，只需Student声明的共有部分包含下面的using声明：

```cpp
using std::valarray<double>::operator[];
```

using 声明只适用于继承，而不适用于包含。


## 14.3 多重继承

### 14.3.1 有多少Worker

假设首先从Singer和Waiter共有派生SingerWaiter：
```cpp
class SingerWaiter: public Singer, public Waiter {...};
```
因为Singer和Waiter都继承了一个Worker组件，因此SingerWaiter将包含两个Worker组件，出现二义性：
```cpp
SingerWaiter ed;
Worker * pw = &ed;  // ambiguous
```

通常，这种赋值将把积累指针设置为派生对象中的基类对象的地址。
```cpp
Worker * pw1 = (Waiter *) &ed;   // the worker in waiter
Worker * pw2 = (Singer *) &ed;   // the worker in Singer
```

#### 1. 虚基类 (virrual base class)
虚基类使得从多个类（他们的基类相同）派生出的对象之继承一个基类对象。例如，通过在类声明中使用关键字virtual，可以使得Worker被用作Singer和Waiter的虚基类（virtual和public的次序无关紧要）：
```cpp
class Singer : virtual public Worker {...};
class Waiter : public virtual Worker {...};
```

然后，可以将SingingWaiter类定义为：
```cpp
class SingingWaiter : public Singer, public Waiter {...};
```
现在，SingingWaiter对象将只包含一个Worker对象的副本。从本质上说，继承的Singer和Waiter对象共享一个Worker对象，而不是各自引入自己的Worker对象副本。

#### 2. 新的构造函数规则：
如果Worker是虚基类，需要使用一下MI构造函数；如果不是虚基类，则是非法的。
```cpp
SingingWaiter(const Worker & wk, int p = 0, int v = Singer::other)
                    : Worker(wk), Waiter(wk, p), Singer(wk, v) {}
```

### 14.3.2 哪个方法
假设没有在SingingWaiter类中重新定义`Show()`方法，并试图使用`SingingWaiter`对象调用继承的`Show()`方法，会导致二义性：
```cpp
SingingWaiter newhire("Elise Hawks", 2005, 6, soprano);
newhire.Show();   // ambiguous
```

需要使用作用域解析符来澄清编程者的意图：
```cpp
SingingWaiter newhire("Elise Hawks", 2005, 6, soprano);
newhire.Singer::Show();
```

### 14.3.3 MI 小结


## 14.4 类模板

### 14.4.1 定义类模板

### 14.4.2 使用模板类

### 14.4.3 深入探讨模板类

### 14.4.4 数组模板示例和非类型参数

### 14.4.5模板多功能性

---

### 14.4.6 模板的具体化

### 14.4.7 成员模板

### 14.4.8 将模板用作参数

### 14.4.9 模板类和友元

### 14.4.10 模板别名（C++11）

---